
# Capital Trades Buy/Sell Data Scrape

In [1]:
import pandas as pd 
from requests_html import HTMLSession
import os
import requests
from lxml import html 

In [2]:
session = HTMLSession()
url = 'https://app.capitoltrades.com/trades?page=1&pageSize=20'
r = session.get(url)
print(r)

<Response [200]>


In [3]:
table = r.html.find('table')[0]
num_poli = 10
rows = table.find('tr')[1:num_poli + 1] # first 'num_poli' items in table, top is header
full_output = ''
# sum all rows to create one block of text from HTML
for i in rows:
    full_output = full_output + i.text
full_output = full_output.replace('\n', '')

In [4]:
# divide chunk of text in table into slices, each corresponding to a single trade
def makeSlices(fullString, endKey):
    start = 0
    end = 0
    slices = []
    while len(fullString) != 0:
        end = fullString.find(endKey) + len(endKey)
        slices.append(
            fullString[start:end]
        )
        fullString = fullString[end:]
    return slices

# exract important trade information given one of the slices made above 
def getTradeInformation(data):
    name = data[
        data.find('Name') + len('Name'):
        data.find('Rep')
    ]
    ticker = data[
        data.find('Ticker') + len('Ticker'):
        data.find('Transaction')
    ]
    transaction = data[
        data.rfind('Transaction') + len('Transaction'):
        data.rfind('Shares')
    ]
    value = data[
        data.find('Value Range') + len('Value Range'):
        data.find('Url')
    ]
    trade_data = {
        'name' : name,
        'ticker' : ticker,
        'transaction type' : transaction,
        'est. value' : value
    }
    return trade_data
    
slices = makeSlices(full_output, 'Url')
for s in slices:
    print(getTradeInformation(s))

{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'GOOGL', 'transaction type': 'Buy (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'GOOGL', 'transaction type': 'Buy (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'CTSH', 'transaction type': 'Sell (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'CTSH', 'transaction type': 'Sell (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'HIG', 'transaction type': 'Sell (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'HIG', 'transaction type': 'Sell (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'GOOGL', 'transaction type': 'Buy (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 'GOOGL', 'transaction type': 'Buy (Stock)', 'est. value': '$1,001 - $15,000'}
{'name': 'Khanna, Rohit (Ro) ', 'ticker': 

In [5]:
# turn value range from string to list of two integers 
def parseValue(valueStr):
    lower, upper = valueStr.split(' - ')
    lower = int(
        ''.join(
            filter(
                lambda x: x.isdigit(), lower
            )
        )
    )
    upper = int(
        ''.join(
            filter(
                lambda x: x.isdigit(), upper
            )
        )
    )
    return [lower, upper]

In [6]:
# value parse example
print(
    parseValue(
    getTradeInformation(
        slices[0]
        )['est. value']
    )
)

[1001, 15000]
